In [ ]:
#!/usr/bin/env python
# -*-coding:utf-8 -*-
'''
  ████
██    ██   Datature
  ██  ██   Powering Breakthrough AI
    ██

@File    :   dicom_nifti_upload.ipynb
@Author  :   Koh Quan Wei Ivan
@Version :   1.0
@Contact :   hello@datature.io
@License :   Apache License 2.0
@Desc    :   Demo for uploading DICOM and NIFTI files using Datature Python SDK.
'''


# Install required libraries

In [1]:
! pip install -U datature

Looking in indexes: https://pypi.org/simple, https://asia-python.pkg.dev/datature-puppeteer/python/simple/


# Import Libraries

In [2]:
from datature.nexus import Client

# Importing DICOM or NIFTI files

Nexus currently has support for the following single file uploads without any need for modification:

1. Single NIFTI files containing slices of a single volume
2. Single DICOM files containing slices 

The following cases will require some modification to the code:

3. Multiple DICOM files in a directory representing a single volume

In [3]:
# change to your own data
nifti_file_path = "data/BRAINIX_NIFTI_Output_3D_File.nii"
dicom_file_path = "data/0009.DCM"
dicom_folder_path = "data/T2"

# Change this to your project secret key to the new project you want to upload your sliced images to
# It can be located on Nexus under the Integrations tab in your project page
SECRET_KEY = "<YOUR_SECRET_KEY"

# Change this to your project ID on Nexus. This can be found via two methods:
# 1. In the URL of the project page (https://nexus.datature.io/project/<YOUR_PROJECT_ID>)
# 2. Project Key in the Integrations page
PROJECT_ID = "proj_<YOUR_PROJECT_ID>"

# Upload DICOM/NIFTI files to Datature Nexus

The steps are as follows:
1. Create a new project on Nexus and retrieve the [secret key](https://developers.datature.io/docs/hub-and-api) in the `Integrations` page.
2. Use Datature SDK to upload the DICOM/NIFTI files to Datature Nexus

For NIFTI files, each file is uploaded as a separate 3D volume.
- If the axis of orientation is provided, the SDK will upload a series of 2D slices corresponding to the specified orientation. You will only see one asset containing multiple slices on Nexus.
- If the axis of orientation is not provided, the SDK will upload a series of 2D slices for each orientation (x, y, z). This means that you will see three separate assets on Nexus.

For DICOM files, only single file uploads are supported. 
- Each single DICOM file can either represent a single image or a series of 2D slices of a 3D volume.
- This means that if you have a 3D volume from multiple stacked DICOM files, you can either:
    - Convert it to a NIFTI file for upload, or 
    - Upload DICOM files consisting of 2D series for each orientation separately.

## Example: Initialize an upload session to upload a NIFTI file (no orientation specified)

Three NIFTI files are uploaded to Datature Nexus, each corresponding to the x, y, and z orientations respectively.

In [4]:
client = Client(SECRET_KEY)
project = client.get_project(PROJECT_ID)

In [5]:
single_nifti_file_path = nifti_file_path

upload_session = project.assets.create_upload_session()

# upload the single NIFTI file containing a 3D volume
with upload_session:
    # this will upload axial, coronal, sagittal planes separately as 3 different assets
    upload_session.add_path(single_nifti_file_path)
upload_session.wait_until_done()
print(f"Uploaded '{nifti_file_path}' to Nexus!")

Uploaded 'data/BRAINIX_NIFTI_Output_3D_File.nii' to Nexus!


## Example: Initialize upload session to upload a NIFTI file (with specified orientation)

Here, we specify that we want to upload the NIFTI file with the orientation of the slices being along the x-axis. You can also specify the orientation to be along the y-axis or z-axis by changing the value of the `nifti_orientation` parameter.

In [7]:
single_nifti_file_path = nifti_file_path

upload_session = project.assets.create_upload_session()

# upload the single NIFTI file containing a 3D volume
with upload_session:
    upload_session.add_path(single_nifti_file_path, nifti_orientation="x")  # x orientation
upload_session.wait_until_done()
print(f"Uploaded '{single_nifti_file_path}' to Nexus!")

Uploaded 'data/BRAINIX_NIFTI_Output_3D_File.nii' to Nexus!


## Example: Initialize an upload session to upload a DICOM file

In [8]:
single_dicom_file_path = dicom_file_path

upload_session = project.assets.create_upload_session()

# upload the single DICOM file containing a series of slices
with upload_session:
    upload_session.add_path(single_dicom_file_path)
upload_session.wait_until_done()
print(f"Uploaded '{single_dicom_file_path}' to Nexus!")

Uploaded 'data/0009.DCM' to Nexus!
